# Generate moment coefficients

The following steps are performed to obtain all the artifacts for force coefficient use case:

1. Read parameters file
2. Read Mesh description
   * Its path can be specified
   * Or it can be obtained from the simulation configuration file
3. Read data for pressure coefficient

In [1]:
from cfdmod.use_cases.pressure.path_manager import CmPathManager
import pathlib

path_manager = CmPathManager(output_path=pathlib.Path("./output/pressure"))

Read parameters file

In [2]:
from cfdmod.use_cases.pressure.moment.Cm_config import CmCaseConfig

config_path = pathlib.Path("./fixtures/tests/pressure/Cm_params.yaml")
post_proc_cfg = CmCaseConfig.from_file(config_path)

post_proc_cfg

CmCaseConfig(bodies={'marquise': BodyConfig(surfaces=['m1_yp', 'm2_zp', 'm3_zm']), 'lanternim': BodyConfig(surfaces=['L1_xp', 'L2_yp', 'L3_zp_yp', 'L4_zp_ym', 'L5_ym', 'L6_xm']), 'building': BodyConfig(surfaces=[])}, moment_coefficient={'measurement_1': CmConfig(body='marquise', sub_bodies=ZoningModel(x_intervals=[-inf, inf], y_intervals=[-inf, inf], z_intervals=[-inf, inf]), variables=['Cmx', 'Cmy', 'Cmz'], lever_origin=(0.0, 10.0, 10.0), statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurtosis']), 'measurement_2': CmConfig(body='building', sub_bodies=ZoningModel(x_intervals=[-inf, inf], y_intervals=[-inf, inf], z_intervals=[-inf, inf]), variables=['Cmx', 'Cmy', 'Cmz'], lever_origin=(0.0, 10.0, 10.0), statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurtosis'])})

Read mesh from file

In [3]:
from nassu.lnas import LagrangianFormat

mesh_path = pathlib.Path("./fixtures/tests/pressure/galpao/galpao.normalized.lnas")
mesh = LagrangianFormat.from_file(mesh_path)

mesh.name, mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

('galpao', (1549, 3), (2915, 3))

Read pressure coefficient data

In [4]:
import pandas as pd

cp_data_path = pathlib.Path("./fixtures/tests/pressure/data/cp_t.resampled.hdf")
cp_data = pd.read_hdf(cp_data_path)

cp_data_to_use = cp_data.to_frame() if isinstance(cp_data, pd.Series) else cp_data
cp_data_to_use

,time_step,point_idx,cp
0,10000.0,0,0.268273
1,10000.0,1,0.273546
2,10000.0,2,0.265319
3,10000.0,3,0.276913
4,10000.0,4,0.272593
...,...,...,...
294410,10100.0,2910,0.485344
294411,10100.0,2911,0.474767
294412,10100.0,2912,0.450817
294413,10100.0,2913,0.537118


Add areas to pressure coefficient data

In [5]:
vec_areas = mesh.geometry._cross_prod() / 2
areas_df = pd.DataFrame({"Ax": vec_areas[:, 0], "Ay": vec_areas[:, 1], "Az": vec_areas[:, 2]})
areas_df["point_idx"] = areas_df.index

cp_data = pd.merge(cp_data, areas_df, on="point_idx", how="left")

Calculate moment coefficient

In [6]:
from cfdmod.use_cases.pressure.moment.Cm_data import process_body
from cfdmod.use_cases.pressure.geometry import get_excluded_surfaces, create_NaN_data_for_mesh
from cfdmod.api.vtk.write_vtk import create_polydata_for_cell_data, merge_polydata

for cfg_label, cfg in post_proc_cfg.moment_coefficient.items():
    body_label = cfg.body
    body_cfg = post_proc_cfg.bodies[body_label]
    processed_body = process_body(mesh=mesh, body_cfg=body_cfg, cp_data=cp_data, cfg=cfg)
    
    sfc_list = [
        sfc for sfc in mesh.surfaces.keys() if sfc not in post_proc_cfg.bodies[body_label].surfaces
    ]
    
    if len(sfc_list) != 0:
        excluded_sfcs = get_excluded_surfaces(mesh=mesh, sfc_list=sfc_list)
        excluded_sfcs.export_stl(path_manager.get_excluded_surface_path(cfg_label))
        # Include polydata with NaN values
        columns = [f"{var}_{statistic}" for statistic in cfg.statistics for var in cfg.variables]
        excluded_data = create_NaN_data_for_mesh(mesh=excluded_sfcs, column_labels=columns)
        excluded_polydata = create_polydata_for_cell_data(data=excluded_data, mesh=excluded_sfcs)
        processed_body.polydata = merge_polydata([processed_body.polydata, excluded_polydata])
        
    processed_body.save_outputs(
        body_label=body_label, cfg_label=cfg_label, path_manager=path_manager
    )